In [5]:
import torch
# import onnx
import torchvision
from gorillatracker.utils.embedding_generator import get_model_for_run_url
import gorillatracker.quantization.quantization_functions as quantization_functions
from gorillatracker.datasets.cxl import CXLDataset
import gorillatracker.quantization.performance_evaluation as performance_evaluation
from gorillatracker.quantization.utils import get_model_input

dataset_path = "/workspaces/gorillatracker/data/splits/ground_truth-cxl-face_images-openset-reid-val-0-test-0-mintraincount-3-seed-42-train-50-val-25-test-25"

model_path = "/workspaces/gorillatracker/quantized_model_weights.pth"
model_wandb_url = "https://wandb.ai/gorillas/Embedding-EfficientNetRWM-CXL-OpenSet/runs/lnw2khtz/workspace"
number_of_calibration_images=100

model = get_model_for_run_url(model_wandb_url, eval_mode=True)
calibration_input_embeddings, _ = get_model_input(
    CXLDataset, dataset_path=dataset_path, partion="train", amount_of_tensors=number_of_calibration_images
)
quantized_model = quantization_functions.ptsq_quantization_fx(model, calibration_input_embeddings)

ParseResult(scheme='https', netloc='wandb.ai', path='/gorillas/Embedding-EfficientNetRWM-CXL-OpenSet/runs/lnw2khtz/workspace', params='', query='', fragment='') ['', 'gorillas', 'Embedding-EfficientNetRWM-CXL-OpenSet', 'runs', 'lnw2khtz', 'workspace'] /gorillas/Embedding-EfficientNetRWM-CXL-OpenSet/runs/lnw2khtz/workspace
Using model from run: 284-train-export-evaluate-EfficientNetRWM-2024-05-14-14-28-06
Config: {'s': 64, 'seed': 42, 'beta1': 0.9, 'beta2': 0.999, 'debug': False, 'end_lr': 1e-06, 'margin': 0.5, 'resume': False, 'compile': False, 'delta_t': 50, 'epsilon': 1e-07, 'l2_beta': 0.01, 'offline': False, 'plugins': None, 'use_ssl': False, 'workers': 16, 'data_dir': '/workspaces/gorillatracker/data/splits/ground_truth-cxl-face_images-openset-reid-val-0-test-0-mintraincount-3-seed-42-train-50-val-25-test-25', 'l2_alpha': 0.1, 'only_val': False, 'profiler': None, 'run_name': '284-train-export-evaluate-EfficientNetRWM', 'start_lr': 0.0001, 'dropout_p': 0.32, 'grad_clip': 1, 'loss_mo

wandb: Downloading large artifact model-lnw2khtz:v1, 984.84MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.9
/opt/conda/envs/research/lib/python3.10/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [6]:
!pip install onnx

In [7]:
# # 1. Convert the model to ONNX https://medium.com/@zergtant/convert-pytorch-model-to-tf-lite-with-onnx-tf-232a3894657c
# import onnx
# input_shape = (1, 3, 224, 224)
# torch.onnx.export(quantized_model, torch.randn(input_shape), "quantized_model.onnx", opset_version=17)

In [9]:
# import onnx_tf
import gorillatracker.quantization.quantization_functions
import gorillatracker.quantization.performance_evaluation as performance_evaluation
import onnx

# Load  ONNX model
onnx_model = onnx.load('quantized_model.onnx')

validations_input_embeddings, validation_labels = get_model_input(
    CXLDataset, dataset_path=dataset_path, partion="val", amount_of_tensors=-1
)

quantized_model_accuracy = performance_evaluation.get_knn_accuracy(
    model=model,
    images=validations_input_embeddings,
    labels=validation_labels,
    device=torch.device("cpu"),
    knn=1,
)
print(f"Quantized model accuracy: {quantized_model_accuracy}")


# Convert ONNX model to TensorFlow format
# tf_model = onnx_tf.backend.prepare(onnx_model)
# # Export  TensorFlow  model 
# tf_model.export_graph("quantized_model.tf")

Quantized model accuracy: {'accuracy': 0.7142857909202576, 'accuracy_top5': 0.7857142686843872, 'auroc': 0.8044486045837402, 'f1': 0.6750332117080688, 'precision': 0.6814877390861511}


/workspaces/gorillatracker/src/gorillatracker/metrics.py:367: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_embeddings = torch.tensor(val_embeddings)
